# Homework 2 - Data from the Web

# Task 1 - QS World Universities ranking

### Importing libs

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt


import requests
from bs4 import BeautifulSoup
import difflib

import json

## Extracting data from the website "topuniversities.com"

The aim of the first task is to scrape meaningful data from _topuniversities.com_ website.
To begin with, we analyze the content and the interaction with the html table on the website. 

![alt text](./images/postman.png "Postman capture")


By analyzing all the REST request loaded by the page we found that the dataset that fills the table is obtained by a GET request to a specific URL in a .txt format.

Analyzing the file we see that the content is stored in a JSON format. Now, we know that we can perform the same request and store the file in a json object that can be used to create our pandas dataframe.

In [2]:
default_path = 'https://www.topuniversities.com'
r = requests.get(default_path +'/sites/default/files/qs-rankings-data/357051.txt?_=1507995842896')
result_json = r.json()

result = pd.DataFrame.from_dict(result_json['data']).head(200)

In [3]:
result

,cc,core_id,country,guide,logo,nid,rank_display,region,score,stars,title,url
0,US,410,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294850,1,North America,100,6,Massachusetts Institute of Technology (MIT),/universities/massachusetts-institute-technolo...
1,US,573,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",297282,2,North America,98.7,5,Stanford University,/universities/stanford-university
2,US,253,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294270,3,North America,98.4,5,Harvard University,/universities/harvard-university
3,US,94,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294562,4,North America,97.7,5,California Institute of Technology (Caltech),/universities/california-institute-technology-...
4,GB,95,United Kingdom,"<a href=""/where-to-study/europe/united-kingdom...","<img src=""https://www.topuniversities.com/site...",294561,5,Europe,95.6,5,University of Cambridge,/universities/university-cambridge
5,GB,478,United Kingdom,"<a href=""/where-to-study/europe/united-kingdom...","<img src=""https://www.topuniversities.com/site...",294654,6,Europe,95.3,5,University of Oxford,/universities/university-oxford
6,GB,365,United Kingdom,"<a href=""/where-to-study/europe/united-kingdom...","<img src=""https://www.topuniversities.com/site...",294014,7,Europe,94.6,None,UCL (University College London),/universities/ucl-university-college-london
7,GB,356,United Kingdom,"<a href=""/where-to-study/europe/united-kingdom...","<img src=""https://www.topuniversities.com/site...",294030,8,Europe,93.7,None,Imperial College London,/universities/imperial-college-london
8,US,120,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294536,9,North America,93.5,5,University of Chicago,/universities/university-chicago
9,CH,201,Switzerland,"<a href=""/where-to-study/europe/switzerland/gu...","<img src=""https://www.topuniversities.com/site...",294432,10,Europe,93.3,None,ETH Zurich - Swiss Federal Institute of Techno...,/universities/eth-zurich-swiss-federal-institu...


## Renaming the columns
In a subsequent task we have to merge this dataframe with another one coming from a different website. So we add the postfix *TU* (for Top Universities) to all column names:

In [ ]:
new_columns =['CC TU','Core_id TU','Country TU','Guide TU','Logo TU','Nid TU','Rank TU','Region TU','Score TU',
             'Stars TU','Title TU','url TU']
result.columns = new_columns

## Scrape details from the details page of each University
In the original JSON object that we saved in our dataframe there is a column called _url_ that specifies the relative path to every University webpage, where we can find the needed details.

Steps to get the details for each University:
1. Request the webpage and parse it with BeautifulSoup.
2. Find the HTML elements that contain the details we need to scrape.
3. It's possible that some Universities don't contain values.
    4. Our approach is to catch possible exceptions and print where we register the error (and link to the website).
4. We add the new values to the main dataframe

In [ ]:
for index, row in result.iterrows():
        r_ = requests.get(default_path + result.iloc[index]['url TU'])
        soup = BeautifulSoup(r_.text, 'html.parser')
        university = result.iloc[index]['Title TU']
        try:
            total_faculty = soup.find('div', class_='total faculty')
            result.loc[index, 'Staff total number TU'] = int(total_faculty.find('div', class_='number').text.split()[0].replace(',', ''))
        except:
            print(university + ' has no values for TOTAL_STAFF') 
            print("Check at: " + default_path + result.iloc[index]['url TU'])
        try:
            total_student = soup.find('div', class_='total student')
            result.loc[index, 'Students total number TU'] = int(total_student.find('div', class_='number').text.split()[0].replace(',', ''))
        except:
            print(university + ' has no values for TOTAL_STUDENTS')
            print("Check at: " + default_path + result.iloc[index]['url TU'])
        try:
            inter_faculty = soup.find('div', class_='inter faculty')
            result.loc[index, 'International Staff number TU'] = int(inter_faculty.find('div', class_='number').text.split()[0].replace(',', ''))
        except:
            print(university + ' has no values for INTERNATIONAL_STAFF')
            print("Check at: " + default_path + result.iloc[index]['url TU'])
        try:
            inter_student = soup.find('div', class_='total inter')
            result.loc[index, 'International Students number TU'] = int(inter_student.find('div', class_='number').text.split()[0].replace(',', ''))
        except:
            print(university + ' has no values for INTERNATIONAL_STUDENTS')
            print("Check at: " + default_path + result.iloc[index]['url TU'])

New York University (NYU) has no values for TOTAL_STAFF
Check at: https://www.topuniversities.com/universities/new-york-university-nyu
New York University (NYU) has no values for TOTAL_STUDENTS
Check at: https://www.topuniversities.com/universities/new-york-university-nyu
New York University (NYU) has no values for INTERNATIONAL_STAFF
Check at: https://www.topuniversities.com/universities/new-york-university-nyu
New York University (NYU) has no values for INTERNATIONAL_STUDENTS
Check at: https://www.topuniversities.com/universities/new-york-university-nyu
Indian Institute of Science (IISc) Bangalore has no values for INTERNATIONAL_STAFF
Check at: https://www.topuniversities.com/universities/indian-institute-science-iisc-bangalore


### Dealing with NaN values

We choose to store absent values as NaN, in order to not modify relations between countries and regions.

### Fixing duplicate ranks
In the first dataframe it is possible to have universities with equal rank. We decided to break those ties before we start the analysis using the order in which the universities are written on the website.

In [ ]:
result['Rank TU'] = range(1,201)

### Removing useless columns
We remove the columns that we consider meaningless for our analysis goals from the dataset:

In [ ]:
results = result.copy()
useless_columns = ['Score TU','Stars TU','CC TU','Core_id TU','Guide TU','Nid TU','url TU','Logo TU']
for column_name in useless_columns:
    del result[column_name]

In [ ]:
df = result.copy()

## Extracting ratios between new values
With the new columns that we have scraped from the website we can produce the ratio between:
- Staff total number / Students total number
- International students number / Students total number

In [ ]:
df['Ratio Staff/Student TU']=df['Staff total number TU']/df['Students total number TU']
df['Ratio International Students TU']=df['International Students number TU']/df['Students total number TU']

### Sorting dataframes by ratios
We sort the dataframes by ratios:

In [ ]:
df_international = df.sort_values('Ratio International Students TU', ascending=0)
df_staffstud = df.sort_values('Ratio Staff/Student TU', ascending=0)

In [ ]:
df_international.head(3)

In [ ]:
df_staffstud.head(3)

### Deleting useless columns before grouping by Country and Region
We delete the useless columns before grouping:

In [ ]:
df_international_by_country_or_region = df_international.copy()
other_useless_columns = ['Rank TU','Ratio International Students TU','Ratio Staff/Student TU','Title TU']
for useless_column in other_useless_columns:
    del df_international_by_country_or_region[useless_column]
df_international_by_country_or_region.head(5)

### Extracting the number of Universities classified in the ranking, grouped by Country
For further analysis in Task 4 we store how many universities are in the top 200 universities, grouped by country:

In [ ]:
df_international_by_country_count = df_international_by_country_or_region.groupby(['Country TU']).count()
col_totals = ['Staff total number TU','Students total number TU','International Staff number TU','International Students number TU']
for c in col_totals:
    del df_international_by_country_count[c]
df_international_by_country_count.columns = ['Number of Universities per country']
df_international_by_country_count = df_international_by_country_count.sort_values('Number of Universities per country', ascending=0)

## Grouping by country and Region

In [ ]:
df_international_by_country = df_international_by_country_or_region.groupby(['Country TU']).sum()
df_international_by_region = df_international_by_country_or_region.groupby(['Region TU']).sum()

### Producing ratios by country

In [ ]:
df_international_by_country['Ratio Staff/Student TU'] = df_international_by_country['Staff total number TU']/df_international_by_country['Students total number TU']
df_international_by_country['Ratio International Students TU'] = df_international_by_country['International Students number TU']/df_international_by_country['Students total number TU']

In [ ]:
df_international_by_country_staffstudent = df_international_by_country.copy()
df_ics = df_international_by_country_staffstudent.sort_values('Ratio Staff/Student TU', ascending=0).head(25)

In [ ]:
df_international_by_country_intstudent = df_international_by_country.copy()
df_ici = df_international_by_country_intstudent.sort_values('Ratio International Students TU', ascending=0).head(25)

### Producing ratios by region

In [ ]:
df_international_by_region['Ratio Staff/Student TU'] = df_international_by_region['Staff total number TU']/df_international_by_region['Students total number TU']
df_international_by_region['Ratio International Students TU'] = df_international_by_region['International Students number TU']/df_international_by_region['Students total number TU']

#### Ratios STAFF/STUDENT

In [ ]:
df_international_by_region_staffstudent = df_international_by_region.copy()
df_irs = df_international_by_region_staffstudent.sort_values('Ratio Staff/Student TU', ascending=0).head(5)

#### Ratios INTERNATIONAL/TOTAL students

In [ ]:
df_international_by_region_intstudent = df_international_by_region.copy()
df_iri = df_international_by_region_intstudent.sort_values('Ratio International Students TU', ascending=0).head(5)

### Extracting the countries and their corresponding regions to be used  for mapping in task 2 

In [ ]:
country_region = df_international.filter(['Country TU','Region TU'], axis=1)

In [ ]:
unique_country_region = country_region.drop_duplicates(subset = 'Country TU')

In [ ]:
country_region_dict = unique_country_region.set_index('Country TU')['Region TU'].to_dict()

In [ ]:
country_region_dict

### Cross comparing this list with all the countries in Task 2 dataframe, only 2 countries are missing:
    1- Russian Federation - > Europe
    2- Luxembourg - > Europe
#### Now we need to add them to the dictionary to avoid NaN values in the region field in the Task 2 dataframe.

In [ ]:
country_region_dict['Russian Federation'] = 'Europe'
country_region_dict['Luxembourg'] = 'Europe'

## Plots

We previously saved the best five universities given the four ratios. In this section we plot the dataframes.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=[16,10])
plt.style.use('ggplot')

plot_useless_columns = ['Staff total number TU','Students total number TU','International Staff number TU','International Students number TU']
for useless_column in plot_useless_columns:
    del df_iri[useless_column]
    del df_ici[useless_column]
    del df_ics[useless_column]
    del df_irs[useless_column]

del df_iri['Ratio Staff/Student TU']
del df_ici['Ratio Staff/Student TU']
del df_irs['Ratio International Students TU']
del df_ics['Ratio International Students TU']

df_irs.plot(ax=axes[0,0], kind='barh')
df_ics.plot(ax=axes[0,1], kind='barh')
df_iri.plot(ax=axes[1,0], kind='barh')
df_ici.plot(ax=axes[1,1], kind='barh')
plt.tight_layout()
plt.show()

Those plots show the ratios for countries and regions. Note that the ratios are only based on the universities which made it to the top 200, and that the result therefore might look very different if all universities of a country were taken into account.

# Task 2 - Times Higher Education ranking

## Extracting data from the website "www.timeshighereducation.com"

![alt text](./images/postman_2.png "Postman capture")


By analyzing all the REST request loaded by the page we found that the dataset that fills the table is obtained by a GET request to a specific URL in a .json format.

In [ ]:
default_path_2 = 'https://www.timeshighereducation.com'
r_2 = requests.get(default_path_2 +'/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json')
result_json_2 = r_2.json()

result_2 = pd.DataFrame.from_dict(result_json_2['data']).head(200)

In [ ]:
result_2.head()

## Deleting useless columns

Removing columns that are of no use to us.

In [ ]:
useless_columns_2 = ['aliases','rank_order','scores_overall_rank','scores_teaching_rank','scores_research_rank',
                     'scores_citations','scores_citations_rank','scores_industry_income','scores_industry_income_rank',
                    'scores_international_outlook','scores_international_outlook_rank','record_type','member_level',
                    'nid','subjects_offered','url']
for column_name in useless_columns_2:
    del result_2[column_name]

In [ ]:
result_2.head()

### Using similar column names as task 1
Every column header extracted from TimesHigherEducation website will contain the word THE 

In [ ]:
new_columns =['Country THE', 'Title THE', 'Rank THE','Scores overall THE', 'Score research THE', 'Score teaching THE',
       'Ratio Female/Male THE', 'Students total number THE',
       'Ratio of International students THE', 'Ratio Student/Staff THE']
result_2.columns = new_columns

### Changing string type into int/float type

In [ ]:
result_2.dtypes

### Some of the data needs to be extracted from existing columns
    1- The total number of international students is computed by multiplying the the ratio of international students and the number of total students.
    2- The Staff total number is computed by dividing the total number of students by the student/staff ratio.

In [ ]:
for i in range(len(result_2)):
    result_2['Ratio of International students THE'][i] = float(result_2['Ratio of International students THE'][i].strip('%'))/100
    result_2['Ratio Student/Staff THE'][i] = float(result_2['Ratio Student/Staff THE'][i])
    result_2['Students total number THE'][i] = int(result_2['Students total number THE'][i].split()[0].replace(',', ''))

result_2['Ratio of International students THE'] = result_2['Ratio of International students THE'].astype('float')
result_2['Ratio Student/Staff THE'] = result_2['Ratio Student/Staff THE'].astype('float')
result_2['International Students number THE'] = (result_2['Ratio of International students THE']*result_2['Students total number THE']).astype('int') 
result_2['Students total number THE'] = result_2['Students total number THE'].astype('int')
result_2['Staff total number THE'] = (result_2['Students total number THE']/result_2['Ratio Student/Staff THE']).astype('int')
result_2['Score research THE'] = result_2['Score research THE'].astype('float')
result_2['Score teaching THE'] = result_2['Score teaching THE'].astype('float')
result_2['Scores overall THE'] = result_2['Scores overall THE'].astype('float')
result_2.dtypes

#Note thet the type of the rank will be changed later.

## Fixing duplicate ranks by reassembling the order
Some of the universities in our ranking are in a tie, we resolve it by using the order defined in the webpage

In [ ]:
result_2['Rank THE'] = range(1,201)

### Below we will answer the first question by comparing the ratio between faculty members and students alongside the ratio of international students for each university

The ratio of international students is already extracted from the JSON.

## Ratio of international students
Sorting the dataframe by ratio of international students and displaying first 5 elements.

In [ ]:
df_international_2 = result_2.sort_values('Ratio of International students THE', ascending=0)

In [ ]:
df_international_2.set_index('Title THE').head()

## Ratio of staff to students

The ratio between faculty members and students is just the inverse of the ratio between students and faculty members.

In [ ]:
result_2['Ratio Staff/Student THE'] = 1 / result_2['Ratio Student/Staff THE']

In [ ]:
result_2.set_index('Title THE').head()

Sorting the dataframe by ratio of Staff to students and displaying first 5 elements.

In [ ]:
df_staffstud_2 = result_2.sort_values('Ratio Staff/Student THE', ascending=0)

In [ ]:
df_staffstud_2.set_index('Title THE').head()

## Mapping the countries to their designated regions

Since the region field is nowhere to be found on the timeshighereducation website, we had to create one.
Following up on the extraction of the country to region dictionary done in Task 1 above, we map every country in the dataframe to it's appropriate region.
A previous analysis was made to know which countries were not common in both dataframes, and they were added manually.

In [ ]:
result_2['Region THE'] = result_2['Country THE'].map(country_region_dict)

In [ ]:
result_2.set_index('Title THE').head()

Checking if there are any NaN values in my new column

In [ ]:
result_2['Region THE'].isnull().sum()

## Grouping by country

In [ ]:
df_international_by_country_2 = result_2.copy()

### Getting rid of useless columns for this task

Notice here we removed the ratio of international students and ratio of student/staff, staff/student because we will compute them again after aggregation of values.

In [ ]:
useless_columns_2 = ['Rank THE','Title THE','Ratio of International students THE','Ratio Student/Staff THE','Ratio Staff/Student THE','Score research THE','Score teaching THE','Scores overall THE']
for column_name in useless_columns_2:
    del df_international_by_country_2[column_name]

#### Grouping:

In [ ]:
df_international_by_country_2 = df_international_by_country_2.groupby(['Country THE']).sum()

In [ ]:
df_international_by_country_2.head()

### Calculating the new ratio between staff/student 

In [ ]:
df_international_by_country_staffstudent_2 = df_international_by_country_2.copy()
df_international_by_country_staffstudent_2['Ratio Staff/Student THE'] = (df_international_by_country_staffstudent_2['Staff total number THE']/df_international_by_country_staffstudent_2['Students total number THE']).astype('float')
df_ics_2 = df_international_by_country_staffstudent_2.sort_values('Ratio Staff/Student THE', ascending=0)
df_ics_2.head()

### Calculating the new ratio of international students

In [ ]:
df_international_by_country_intstudent_2 = df_international_by_country_2.copy()
df_international_by_country_intstudent_2['Ratio of International students THE'] = (df_international_by_country_intstudent_2['International Students number THE']/df_international_by_country_intstudent_2['Students total number THE']).astype('float')
df_ici_2 = df_international_by_country_intstudent_2.sort_values('Ratio of International students THE', ascending=0)
df_ici_2.head()

## Grouping by Region

In [ ]:
df_international_by_region_2 = result_2.copy()

### Getting rid of useless columns for this task

Notice here we removed the ratio of international students and ratio of student/staff, staff/student because we will compute them again after aggregation of values.

In [ ]:
useless_columns_2 = ['Rank THE','Title THE','Country THE','Ratio of International students THE','Ratio Student/Staff THE','Ratio Staff/Student THE','Score research THE','Score teaching THE','Scores overall THE']
for column_name in useless_columns_2:
    del df_international_by_region_2[column_name]

#### Grouping:

In [ ]:
df_international_by_region_2 = df_international_by_region_2.groupby(['Region THE']).sum()

### Calculating the new ratio between staff/student 

In [ ]:
df_international_by_region_staffstudent_2 = df_international_by_region_2.copy()
df_international_by_region_staffstudent_2['Ratio Staff/Student THE'] = (df_international_by_region_staffstudent_2['Staff total number THE']/df_international_by_region_staffstudent_2['Students total number THE']).astype('float')
df_irs_2 = df_international_by_region_staffstudent_2.sort_values('Ratio Staff/Student THE', ascending=0)
df_irs_2.head()

### Calculating the new ratio of international students

In [ ]:
df_international_by_region_intstudent_2 = df_international_by_region_2.copy()
df_international_by_region_intstudent_2['Ratio of International students THE'] = (df_international_by_region_intstudent_2['International Students number THE']/df_international_by_region_intstudent_2['Students total number THE']).astype('float')
df_iri_2 = df_international_by_region_intstudent_2.sort_values('Ratio of International students THE', ascending=0).head(5)
df_iri_2.head()

## Plots

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=[16,10])
plt.style.use('ggplot')

df_irs_2 = df_irs_2.filter(['Region THE','Ratio Staff/Student THE'],axis=1)
df_ics_2 = df_ics_2.filter(['Country THE','Ratio Staff/Student THE'],axis=1)
df_iri_2 = df_iri_2.filter(['Region THE','Ratio of International students THE'],axis=1)
df_ici_2 = df_ici_2.filter(['Country THE','Ratio of International students THE'],axis=1)

df_irs_2.plot(ax=axes[0,0], kind='barh')
df_ics_2.plot(ax=axes[0,1], kind='barh')
df_iri_2.plot(ax=axes[1,0], kind='barh')
df_ici_2.plot(ax=axes[1,1], kind='barh')
plt.tight_layout()
plt.show()

# Task 3 - Merging the dataframes

The next goal is to merge the two dataframes that were created in the previous tasks using university names.

We first observe that a straight forward approach based on equality of the names is insufficient, because there might be slight differences in the names. For example, the MIT is referred to as 'Massachusetts Institute of Technology (MIT)' in the Top Universities ranking, while it referred to as 'Massachusetts Institute of Technology' in the Times Higher Education ranking. 

Therefore, we need a more sophisticated method for matching names. We decided to first compute all possible combinations of universities, i.e. the cross product of the two dataframes, and then filter on a refined criterion. For every combination of universities a metric for the similarity between their titles is computed using the `ratio` function provided by the difflib Python standard library. The core functionality of this library is to calculate matching parts in sequences. This is achieved by searching for the longest common subsequence and then recursivly looking for matches left and right of the occurence of this sequence. (This is based on Gestalt pattern matching<sup>1</sup>). 
The ratio of similarity is then computed using the formula 2.0 * M / T where M is the number of matches and T is the sum of the lengths of the two strings. This results in a number between 0 and 1 where 1 means that the two strings are equal. We then define a threshold for the point from which on two names are considered to describe the same university. We set this threshold to be 0.8. The threshold was found by manually checking the quality of the matching.

This approach works well apart from a few exceptions. In particular, there were some mistakes for titles which consist mainly of words which are very common in university names. For example the names 'University of Bern' and 'University of Bonn' would match. Therefore, we decided to further refine our method by deleting those common words from the titles before computing the ratios. We delete the words 'University', 'Institute', 'of', and 'The'.

As it is difficult to create a perfect approximation, there still are some entries in the end which need to be added or deleted manually.  In total the result has 144 rows, where two rows had to be deleted manually and five rows were added manually. We consider this a satisfactory accuracy. 

<sup>1</sup> John W. Ratcliff and David Metzener, Pattern Matching: The Gestalt Approach, Dr. Dobb's Journal, page 46, July 1988. 





#### Removing useless columns from a copy of the original dataframe

In [ ]:
useless_columns_m = ['Stars TU','CC TU','Core_id TU','Guide TU','Nid TU','url TU','Logo TU']
for column in useless_columns_m:
    del results[column]

### Deleting misleading words:

In [ ]:
dict_words = {'University':'','of':'','The':'','Institute':''}

In [ ]:
uni_names_2 = list(result_2['Title THE'])
stripped_names_2 = []
for uni in uni_names_2:
    for i, j in dict_words.items():
        uni = uni.replace(i,j)
    uni=uni.split()
    uni = ' '.join(uni)
    stripped_names_2.append(uni)
    
    

In [ ]:
uni_names_1 = list(results['Title TU'])
stripped_names_1 = []
for uni in uni_names_1:
    for i, j in dict_words.items():
        uni = uni.replace(i,j)
    uni=uni.split()
    uni = ' '.join(uni)
    stripped_names_1.append(uni)

In [ ]:
results['Stripped Names TU'] = stripped_names_1

In [ ]:
result_2['Stripped Names THE'] = stripped_names_2

In [ ]:
results.head()

### Setting the Overall Score in THE dataframe to the same range as the TU frame for fair comparison further on.

In [ ]:
results['Score TU'] = results['Score TU'].astype('float')
OldRange = (max(result_2['Scores overall THE']) - min(result_2['Scores overall THE']))  
NewRange = (max(results['Score TU']) - min(results['Score TU']))
result_2['Scores overall THE'] =  (((result_2['Scores overall THE'] - min(result_2['Scores overall THE'])) * NewRange) / OldRange) +  min(results['Score TU'])                       

In [ ]:
result_2.head()

### Adding similarity metric and filtering:

In [ ]:
from difflib import SequenceMatcher
df_cartesian = pd.merge(results.assign(key=0), result_2.assign(key=0), on='key').drop('key', axis=1)

def similar(row):
    return SequenceMatcher(None, row['Stripped Names TU'],row['Stripped Names THE']).ratio()

df_cartesian['Similarity'] = df_cartesian.apply(similar,axis=1)
df_merged = df_cartesian[df_cartesian.Similarity > 0.80]
df_merged.head()

### Extracting manually the indices of mismatches:

False positives can easily be detected by checking the resulting dataframe. We found two problematic rows, one of them matching 'Northwestern' and 'Northeastern', the other one matching 'Hong Kong Science and Technology' and 'Pohang Science and Technology' (considering the stripped names).

This corresponds to rows with the following information:

In [ ]:
#['Northwestern', 'Northeastern', 0.9166666666666666] index: 5792
#['Hong Kong Science and Technology','Pohang Science and Technology', 0.8524590163934426], index : 5936

The indices of the mismatching entries where retrieved as follows:

In [ ]:
print(df_cartesian[ (df_cartesian['Stripped Names TU'].str.startswith('Northwestern')) & (df_cartesian['Stripped Names THE'].str.startswith('Northeastern'))].index[0])
print(df_cartesian[ (df_cartesian['Stripped Names TU'].str.startswith('Hong Kong Science and Technology')) & (df_cartesian['Stripped Names THE'].str.startswith('Pohang Science and Technology'))].index[0])

For detecting false negatives, we took a look at rows with match results between 0.65 and 0.8:

In [ ]:
df_cartesian[ (df_cartesian['Similarity'] > 0.65)  & (df_cartesian['Similarity'] <= 0.8)].head()

We found that the rows with the following information should be added:

In [ ]:
#['New York (NYU)', 'New York', 0.7272727272727273], index:10227
#['UCL ( College London)', 'College London ucl', 0.717948717948718], index : 1215
#['KAIST - Korea Advanced Science & Technology','Korea Advanced Science and Technology (KAIST)',0.7727272727272727],
#index : 8095
#['Humboldt-Universität zu Berlin', 'Humboldt Berlin', 0.6666666666666666]. index : 23861
#['New South Wales (UNSW Sydney)', 'New South Wales', 0.6818181818181818], index : 8884

Again, we retrieved the corresponding indices:

In [ ]:
print(df_cartesian[ (df_cartesian['Stripped Names TU'].str.startswith('New York (NYU)')) & (df_cartesian['Stripped Names THE'].str.startswith('New York'))].index[0])
print(df_cartesian[ (df_cartesian['Stripped Names TU'].str.startswith('UCL ( College London)')) & (df_cartesian['Stripped Names THE'].str.startswith('College London'))].index[0])
print(df_cartesian[ (df_cartesian['Stripped Names TU'].str.startswith('KAIST - Korea Advanced Science & Technology')) & (df_cartesian['Stripped Names THE'].str.startswith('Korea Advanced Science and Technology (KAIST)'))].index[0])
print(df_cartesian[ (df_cartesian['Stripped Names TU'].str.startswith('Humboldt-Universität zu Berlin')) & (df_cartesian['Stripped Names THE'].str.startswith('Humboldt Berlin'))].index[0])
print(df_cartesian[ (df_cartesian['Stripped Names TU'].str.startswith('New South Wales (UNSW Sydney)')) & (df_cartesian['Stripped Names THE'].str.startswith('New South Wales'))].index[0])

### Adding and removing the mismatches:

In [ ]:
adding_index = [10227,1215,8095,23861,8884]
removing_index = [5792,5936]

In [ ]:
for index in removing_index:
    df_merged = df_merged.drop(index)

In [ ]:
for index in adding_index:
    df_merged = df_merged.append(df_cartesian.iloc[index])

Before we proceed to the next task, we drop the columns which were just needed for the merging and print the final result.

In [ ]:
del df_merged['Similarity']
del df_merged['Stripped Names THE']
del df_merged['Stripped Names TU']
df_merged.head()

# Task 4 - Exploratory Analysis 

To begin with, we try to find correlations between the different variables by computing the pairwise correlation of the columns. If called without additional arguments, the pandas method 
`corr` computes the standard (Pearson) correlation coefficients. That is, coefficients between -1 and 1 where 1 represents total positive linear correlation, 0 represents no correlation and -1 represents total negative linear correlation. This gives us a good starting point to come up with interesting questions for the further analysis.

In [ ]:
df_merged.corr()

In the rest of our analysis we examine the following questions:
1. When a university is strong in its international dimension, can you observe a consistency both for students and faculty members? (see example in exercise description.)
2. Do international students tend to chose higher raked universities?
3. How similar are the ranks assigned by the two rankings?
4. Is the performance in research related to the performance in teaching?
5. 5. Is the number of universities in the top 200 in a country correlated to:
    - GDP of the country?
    - Billions of USD invested in R&D by the country?

### 1. When a university is strong in its international dimension, can you observe a consistency both for students and faculty members?

In order to answer this question, we consider the correlation between the values 'International Staff number TU' and 'International Students number TU' computed above. Furthermore, we draw a scatter plot to get further insights into the relationship. Note that the analysis is based on the Top Universities ranking. 

In [ ]:
df_merged.plot.scatter(x='International Staff number TU', y='International Students number TU')

*Interpretation:* Between those indicators there exists a medium positive linear relationship, indicating that the number of international staff also increases the number of international students. However, it might be that there is an underlying factor influencing both, e.g. university size, location, teached subjects or international ranking.

### 2. Do international students tend to chose higher raked universities?

For this question we take a closer look at the rank and the ratio of international students provided in the Times Higher Education ranking. We again support our analysis by a scatter plot: 

In [ ]:
df_merged.plot.scatter(x='Rank THE', y='Ratio of International students THE')

*Interpretation:* There is a small negative linear correlation. Note that a smaller rank is higher up, i.e. better. Therefore a negative correlation means that universities higher up in the ranking have a higher ratio of international students. Closer inspection of the scatter plot indicates that this relationship is influenced by outliers, universities that have a high number of international students and small rank. Most noticeable, the London School of Economics and Political Science has a rank of 25 and 71% of international students. Ignoring those outliers, no relationship between these two indicators could be detected.

### 3. How similar are the ranks assigned by the two rankings?

We use the same approach as above to compare the ranks:

In [ ]:
df_merged.plot.scatter(x='Rank TU', y='Rank THE')

*Interpretation:* These variables exhibit a positive medium linear correlation. Especially, universities higher up, i.e. with small rank, in the Top Universities ranking also are higher up in the Times Higher Education ranking, whereas variation in creases with higher rank. There are quite a few universitites where there is a clear discrepancy between the ranks. For example, the University of Minnesota has rank 163 in the Top Universities ranking and rank 56 in the Times Higher Education ranking. A further analysis of this example could provide insights into the different approaches for the rank computation.

### 4. Is the performance in research related to the performance in teaching?

For this analysis we compare the reserch and the teaching score provided by the Times Higher Education ranking.

In [ ]:
df_merged.plot.scatter(x='Score research THE', y='Score teaching THE')

*Interpretation:* These scores are nearly perfectly (positive) linearly correlated, meaning that universities with a high research score show a nearly equally high teaching score. As we do not know how exactly those scores are computed, it is difficult to draw concrete conclusions from this result. Interestingly, as can be concluded from the following scatter plot, there is no clear relationship between the teaching score and the ratio between students and staff.

In [ ]:
df_merged.plot.scatter(x='Score teaching THE', y='Ratio Student/Staff THE')

### 5. Is the number of universities in the top 200 in a country correlated to:
1. GDP of the Country?
2. Billions of USD invested in R&D?

For this analysis we compare the number of universities in the best 200 universities for each country with statistics of GDP per country and billions of USD invested in R&D per country. We also consider GDP and money invested in R&D per-capita.

Since these information is not given on the ranking websites, we scrape this content from three Wikipedia pages:
- https://en.wikipedia.org/wiki/List_of_countries_by_research_and_development_spending
- https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)
- https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)_per_capita

In [ ]:
from bs4 import BeautifulSoup

import urllib.request as urllib2
wiki = "https://en.wikipedia.org/wiki/List_of_countries_by_research_and_development_spending"
wiki2 = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)'
wiki3 = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)_per_capita'

header = {'User-Agent': 'Mozilla/5.0'}
req = urllib2.Request(wiki,headers=header)
req2 = urllib2.Request(wiki2,headers=header)
req3 = urllib2.Request(wiki3,headers=header)

page = urllib2.urlopen(req)
page2 = urllib2.urlopen(req2)
page3 = urllib2.urlopen(req3)

soup = BeautifulSoup(page, "lxml")
soup2 = BeautifulSoup(page2, "lxml")
soup3 = BeautifulSoup(page3, "lxml")


country_ef = []
ranking_ef = []
pro_gdp = []

country_gdp = []
ranking_gdp = []

country_pro = []
ranking_pro = []

table = soup.find("table", { "class" : "wikitable sortable" })
table2 = soup2.find("table", { "class" : "wikitable sortable" })    
table3 = soup3.find("table", { "class" : "wikitable sortable" })    

for row in table3.findAll("tr"):
    cells = row.findAll("td")
    if len(cells) == 3:
        country_pro.append(cells[1].find('a').find(text=True))
        ranking_pro.append(cells[2].findAll(text=True)[0])
        
for row in table2.findAll("tr"):
    cells = row.findAll("td")
    if len(cells) == 3:
        country_gdp.append(cells[1].find('a').find(text=True))
        ranking_gdp.append(cells[2].findAll(text=True)[0])
        
for row in table.findAll("tr"):
    cells = row.findAll("td")
    if len(cells) == 7:
        country_ef.append(cells[1].find('a').find(text=True))
        ranking_ef.append(cells[2].findAll(text=True)[0])
        pro_gdp.append(cells[4].findAll(text=True)[0])
    
df_international_by_country_count2 = df_international_by_country_count.copy()

for i, c in enumerate(df_international_by_country_count.index):
    if c in country_gdp:
        df_international_by_country_count2.loc[c, 'GDP per country'] = float(ranking_gdp[country_gdp.index(c)].replace(',',''))
    if c in country_ef:
        df_international_by_country_count2.loc[c, 'Billions of dollars invested in R&D'] = float(ranking_ef[country_ef.index(c)].replace(',',''))
        df_international_by_country_count2.loc[c, 'Dollars per-capita invested in R&D'] = float(pro_gdp[country_ef.index(c)].replace(',',''))
    if c in country_pro:
        df_international_by_country_count2.loc[c, 'GDP per-capita per country'] = float(ranking_pro[country_pro.index(c)].replace(',','').replace('%',''))

        
last_corr_df = df_international_by_country_count2.fillna(95.00).astype(float)
axs1 = last_corr_df.head(100).plot(subplots=True, kind='bar', figsize=[16,10], sharex=True)
axs1[0].set_ylim(0,50)
axs1[0].set_ylabel('Universities in top 200')
axs1[1].set_ylim(100000,8000000)
axs1[1].set_ylabel('Billions US$ invested in R&D')
axs1[2].set_ylim(0, 500)
axs1[2].set_ylabel('GDP US$')

We decided to also do a more thorough analysis. As before we compute the correlation matrix and draw scatter plots.

In [ ]:
last_corr_df.corr()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=[16,10])

last_corr_df.plot.scatter(ax=axes[0][0],x='GDP per country',y='Number of Universities per country')
last_corr_df.plot.scatter(ax=axes[0][1],x='Billions of dollars invested in R&D',y='Number of Universities per country')
last_corr_df.plot.scatter(ax=axes[0][2],x='GDP per country',y='Billions of dollars invested in R&D')

last_corr_df.plot.scatter(ax=axes[1][0],x='GDP per-capita per country',y='Number of Universities per country')
last_corr_df.plot.scatter(ax=axes[1][1],x='Dollars per-capita invested in R&D',y='Number of Universities per country')
last_corr_df.plot.scatter(ax=axes[1][2], x='GDP per-capita per country',y='Dollars per-capita invested in R&D')

*Interpretation:* To begin with, we consider the relationship between the number of Top 200 universities and the GDP per country. A strong positive linear relationship is given (rho = 0.7889) from which we might draw the conclusion that quality of universities increases with a country's wealth. However, the GPD per capita draws an entirely different picture. Considering this normalized indicator of a country's wealth it is clear that the number of Top 200 universities within a country is only mildly correlated with its wealth (rho = 0.2997).

Next, we consider the relationship between the number of Top 200 universities and the R&D per country. A strong positive linear relationship is given (rho = 0.6846) from which we might draw the conclusion that quality of universities increases with a country's R&D expenses. However, again the R&D per capita draws an entirely different picture. Considering this normalized indicator of a country's R&D expenses it is clear that the number of Top 200 universities within a country is only mildly correlated with its wealth (rho = 0.2546). We therefore conclude that money invested or a countries wealth only mildly influence the universities' quality.

# Task 5 - Finding the best university 

### Looking at the criteria www.topuniversities.com use to rank the universities it is divided in this way :
    1-Academic reputation (40%)
    2-Employer reputation (10%)
    3-Faculty/Student Ratio (20%)
    4-Citations per faculty (20%)
    5-International faculty ratio/International student ratio (5% each)

### Looking at the criteria www.timeshighereducation.com use to rank the universities it is divided in this way :
    1- Teaching - the learning environment (30%)
        -Reputation Survey (15%)
        -Staff to student ratio (4.5%)
        -Doctorate-to-bachelor's ratio (2.25%)
        -Doctorates awarded to academic staff ratio (6%)
        -Institutional Income (2.25%)
    2- Research - Volume, income, and reputation (30%)
        -Reputation Survey (18%)
        -Research income (6%)
        -Research Productivity (6%)
    3- Citations - research influence (30%)
    4- International outlook (7.5%)
        -International to domestic staff ratio (2.5%)
        -International to domestic student ratio (2.5%)
        -International collaboration (2.5%)
    5- Industry income - Knowledge transfer ( 2.5% )

In [ ]:
comp_df = df_merged.filter(['Rank TU','Rank THE','Country TU','Score TU','Scores overall THE','Title TU','Students total number THE','Students total number TU','International Students number TU','International Students number THE','Ratio Staff/Student THE','Ratio Staff/Student TU','Score research THE','Score teaching THE','Ratio Student/Staff THE'],axis=1)

In [ ]:
comp_df.set_index('Title TU').head(6)

#### Excluding other universities the competition for the best university is between the 6 universities listed up, where both ranking systems show that they should be on top 6.
#### Taking into consideration the criteria of ranking listed above, we can see that the TU ranking system contributes 60% of its total to Academic reputation + Citations per faculty, while THE ranking system contributes 90% of its total to Teaching + Research + Citation.
#### Oxford University would be listed as the best university. Checking the dataframe above it is leading in research score and 3rd highest teaching score. Although the high number of students it still maintains the highest student/staff ratio between all of them.
#### Oxford is also the most diverse w.r.t. gender with a  46:54 female to male ratio, and has the highest number of international students. The only reason why Oxford comes 6th in TU ranking system is due to the citation per faculty critereon and we find the division of the Research/Teaching/citation criteria is better constructed in the THE ranking system.